In [1]:
import os
import csv
import time
import requests
import geopandas as gpd
from shapely.geometry import box, Point
from shapely.ops import unary_union

In [2]:
MAPILLARY_ACCESS_TOKEN = "MLY|29607086385604993|05c6b7344dbbb999b79bd9a6ce3a0145"
OUTPUT_DIR = "country_dataset"
CSV_FILENAME = "country_dataset.csv"
IMAGES_PER_COUNTRY = 100
MAX_RETRIES = 3

In [3]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
countries = gpd.read_file("natural_earth/ne_110m_admin_0_countries.shp")

In [5]:
def fetch_mapillary_images_bbox(bbox, limit=100, retries=MAX_RETRIES):
    """Pobiera listę zdjęć z Mapillary w danym bbox.
    bbox = (min_lon, min_lat, max_lon, max_lat)
    """
    url = (
        f"https://graph.mapillary.com/images"
        f"?access_token={MAPILLARY_ACCESS_TOKEN}"
        f"&fields=id,thumb_1024_url,geometry"
        f"&limit={limit}"
        f"&bbox={bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}"
    )
    for attempt in range(retries):
        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            data = r.json()
            return data.get("data", [])
        except Exception as e:
            print(f"  [Błąd API Mapillary, próba {attempt+1}/{retries}] {e}")
            time.sleep(5)
    return []

In [6]:
def download_image(url, path):
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
        with open(path, "wb") as f:
            f.write(r.content)
        return True
    except Exception as e:
        print(f"  Błąd pobierania: {e}")
        return False

In [7]:
with open(CSV_FILENAME, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["country", "latitude", "longitude", "local_path"])

    for idx, country_row in countries.iterrows():
        country_name = country_row["NAME"]
        country_geom = country_row["geometry"]

        print(f"\n🌍 Przetwarzanie kraju: {country_name}")

        # Bounding box kraju
        minx, miny, maxx, maxy = country_geom.bounds
        bbox = (minx, miny, maxx, maxy)

        country_dir = os.path.join(OUTPUT_DIR, country_name.replace(" ", "_"))
        os.makedirs(country_dir, exist_ok=True)

        images = fetch_mapillary_images_bbox(bbox, limit=IMAGES_PER_COUNTRY*2) 
        if not images:
            print(f"  ❌ Brak zdjęć w obszarze bbox kraju {country_name}")
            continue

        saved = 0
        for image in images:
            # Sprawdź, czy punkt znajduje się wewnątrz granicy kraju (nie tylko w bbox)
            coords = image.get("geometry", {}).get("coordinates", None)
            if coords is None:
                continue
            point = Point(coords)
            if not country_geom.contains(point):
                continue

            image_id = image["id"]
            image_url = image.get("thumb_1024_url")
            if not image_url:
                continue

            filename = f"{image_id}.jpg"
            local_path = os.path.join(country_dir, filename)

            if download_image(image_url, local_path):
                lat, lon = point.y, point.x
                writer.writerow([country_name, lat, lon, local_path])
                #print(f" Pobrano {saved+1}: {filename} ({lat:.5f}, {lon:.5f})")
                saved += 1

            if saved >= IMAGES_PER_COUNTRY:
                break
            time.sleep(0.2)

        if saved == 0:
            print(f"  ❌ Nie znaleziono zdjęć wewnątrz granicy kraju {country_name}")

print("\nZakończono tworzenie datasetu!")
print(f"Dane zapisane w: {os.path.abspath(OUTPUT_DIR)} i {os.path.abspath(CSV_FILENAME)}")


🌍 Przetwarzanie kraju: Fiji
  ❌ Nie znaleziono zdjęć wewnątrz granicy kraju Fiji

🌍 Przetwarzanie kraju: Tanzania

🌍 Przetwarzanie kraju: W. Sahara
  ❌ Nie znaleziono zdjęć wewnątrz granicy kraju W. Sahara

🌍 Przetwarzanie kraju: Canada

🌍 Przetwarzanie kraju: United States of America

🌍 Przetwarzanie kraju: Kazakhstan
  ❌ Nie znaleziono zdjęć wewnątrz granicy kraju Kazakhstan

🌍 Przetwarzanie kraju: Uzbekistan

🌍 Przetwarzanie kraju: Papua New Guinea

🌍 Przetwarzanie kraju: Indonesia

🌍 Przetwarzanie kraju: Argentina

🌍 Przetwarzanie kraju: Chile

🌍 Przetwarzanie kraju: Dem. Rep. Congo

🌍 Przetwarzanie kraju: Somalia

🌍 Przetwarzanie kraju: Kenya

🌍 Przetwarzanie kraju: Sudan

🌍 Przetwarzanie kraju: Chad

🌍 Przetwarzanie kraju: Haiti

🌍 Przetwarzanie kraju: Dominican Rep.

🌍 Przetwarzanie kraju: Russia
  [Błąd API Mapillary, próba 1/3] 500 Server Error: Internal Server Error for url: https://graph.mapillary.com/images?access_token=MLY%7C29607086385604993%7C05c6b7344dbbb999b79bd9a6ce3